**Data Ingestion**
* To install jdbc driver, download mariadb jar from mariadb.com
* Copy mariadb jar into $SPARK_HOME/jars/
* In Remote-WSL, change ~/.vscode-server
  1. PATH=/home/nolfonzo/miniconda3/bin:$PATH                                                                                     
  2. export SPARK_HOME="/home/nolfonzo/spark-3.2.1-bin-hadoop3.2"
  3. export PATH="$SPARK_HOME/bin:$SPARK_HOME/sbin:$PATH"
  4. export JAVA_HOME="/usr/lib/jvm/java-11-openjdk-amd64"
  5. echo "**** PATH: " $PATH
  6. echo "**** SPARK_HOME: " $SPARK_HOME
  7. echo "**** JAVA_HOME: " $JAVA_HOME
~                                          

In [9]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#
# use the spark.read() method to load data from a JDBC data source
import pyspark
sc = pyspark.SparkContext.getOrCreate()
from pyspark.sql import SparkSession
spark = SparkSession(sc)
dataframe_mysql = spark.read.format("jdbc").options(
    url="jdbc:mysql://127.0.0.1/pysparkdb",
    driver = "org.mariadb.jdbc.Driver",
    dbtable = "authors",
    user="root",
    password="g0nz07").load()
dataframe_mysql.show()

+---+----------+----------+-----------+
|uid| last_name|first_name|birth_month|
+---+----------+----------+-----------+
|  1|    Asimov|     Issac|    January|
|  2|     Brown|       Dan|       June|
|  3|  Baldacci|     David|     August|
|  4| Burroughs|Edgar Rice|  September|
|  5|    Clarke| Arthur C.|   December|
|  6|      Cook|     Robin|        May|
|  7|   Dickens|   Charles|   February|
|  8|   Forster|      E.M.|    January|
|  9|    Lahiri|    Jhumpa|       July|
| 10|      Rand|       Ayn|   February|
| 11|  Salinger|      J.D.|    January|
| 12|     Seuss|       Dr.|      March|
| 13|    Stoker|      Bram|   November|
| 14|     Wilde|     Oscar|    October|
| 15|Wordsworth|   William|      April|
+---+----------+----------+-----------+



In [4]:
# use the spark.read() function to read a CSV file. We specify the inferSchema and header options to be true. This helps Spark infer the column names and data type information by reading a sample set of data.
retail_df = (spark
    .read
    .format("csv")
    .option("inferSchema", "true")
    .option("header","true")
    .load("/home/nolfonzo/src/pyspark/Essential-PySpark-for-Scalable-Data-Analytics/README.md")
    )
retail_df.show()

+-----------------------------------------------+
|# Essential PySpark for Scalable Data Analytics|
+-----------------------------------------------+
|                           <a href="https://...|
|                           This is the code ...|
|                           **A beginner's gu...|
|                           ## What is this b...|
|                           Apache Spark is a...|
|                           This book covers ...|
|                           Understand the ro...|
|                           Gain an appreciat...|
|                           Scale out your da...|
|                           Build data pipeli...|
|                           Leverage the clou...|
|                           Explore the appli...|
|                           Integrate your cl...|
|                           If you feel this ...|
|                           <a href="https://...|
|                           alt="https://www....|
|                           ## Instructions a...|


In [8]:
# Semi Structured storage formats
# Examples of semi-structured data formats include CSV, XML, and JSON.
# Not suitable for binary types
#
# ingest unstructured data and convert it into a structured format before storing it inside the data lake. This makes the downstream processing of data easier and more efficient.
#
# We load a set of images files using Spark's built-in image format; the result is a Spark DataFrame of image attributes.
raw_df = spark.read.format("image").load("/home/nolfonzo/src/pyspark/Essential-PySpark-for-Scalable-Data-Analytics/data/images")
# We use the printSchema() function to take a look at the DataFrame's schema and discover that the DataFrame has a single nested column named image with origin, height, width, nChannels etc
raw_df.printSchema()
# bring up the inner attributes to the top level using the image prefix with each inner attribute, such as image.origin, 
# and create a new DataFrame named image_df with all of the image's individual attributes as top-level columns.
image_df = raw_df.select("image.origin", "image.height", "image.width", "image.nChannels", "image.mode", "image.data")
image_df.show()
# Now that we have our final DataFrame, we write it out to the data lake using the CSV format.
image_df.write.option("header", "true").mode("overwrite").csv("/home/nolfonzo/src/pyspark/images.csv")

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)

+--------------------+------+-----+---------+----+--------------------+
|              origin|height|width|nChannels|mode|                data|
+--------------------+------+-----+---------+----+--------------------+
|file:///home/nolf...|   279| 1480|        3|  16|[0E 54 72 0D 53 7...|
|file:///home/nolf...|   323| 1504|        3|  16|[57 B4 E1 5A B9 E...|
|file:///home/nolf...|   371| 1530|        3|  16|[FF FF FF FF FF F...|
|file:///home/nolf...|   268|  763|        3|  16|[3F DD B5 3C D7 B...|
|file:///home/nolf...|   197|  699|        3|  16|[FF FF FF FF FF F...|
|file:///home/nolf...|   244|  617|        3|  16|[FF FF FF FF FF F...|
|file:///home/nolf...|   224|  765|        3|  16

In [10]:
# Another way to make header top-level columns
csv_df = (spark \
    .read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/home/nolfonzo/src/pyspark/Essential-PySpark-for-Scalable-Data-Analytics/data/online_retail/online_retail_II_small.csv") \
)
csv_df.show()
retail_df = csv_df.selectExpr("Invoice as InvoiceNo", "StockCode", "Description", "Quantity", "InvoiceDate", "Price as UnitPrice", "`Customer ID` as CustomerID", "Country")
retail_df.show()

+-------+---------+--------------------+--------+--------------+-----+-----------+--------------+
|Invoice|StockCode|         Description|Quantity|   InvoiceDate|Price|Customer ID|       Country|
+-------+---------+--------------------+--------+--------------+-----+-----------+--------------+
| 489434|    85048|15CM CHRISTMAS GL...|      12|01/12/09 07:45| 6.95|      13085|United Kingdom|
| 489434|   79323P|  PINK CHERRY LIGHTS|      12|01/12/09 07:45| 6.75|      13085|United Kingdom|
| 489434|   79323W| WHITE CHERRY LIGHTS|      12|01/12/09 07:45| 6.75|      13085|United Kingdom|
| 489434|    22041|"RECORD FRAME 7""...|      48|01/12/09 07:45|  2.1|      13085|United Kingdom|
| 489434|    21232|STRAWBERRY CERAMI...|      24|01/12/09 07:45| 1.25|      13085|United Kingdom|
| 489434|    22064|PINK DOUGHNUT TRI...|      24|01/12/09 07:45| 1.65|      13085|United Kingdom|
| 489434|    21871| SAVE THE PLANET MUG|      24|01/12/09 07:45| 1.25|      13085|United Kingdom|
| 489434|    21523|F